**1. Set Up API Credentials:**
- Google Generative AI API
- Obtain your API key from the Google AI Studio.
- Store the API key in your Google Colab environment in the 'secrets' section
- Set the secret name as ```'API_KEY'``` with the secret key value which you've created.

**2. Run the Application:**
- Simply run the cells sequentially.

**3. Testing the Application:**
- Enter sample user queries into the application to test its functionality. For Example:

  ```user_query = "Get me Flipkart's GMV for the last one year."```

In [ ]:
!pip install google-generativeai
!pip install dateparser
!pip install python-dotenv

In [ ]:
import os
from google.colab import userdata
import google.generativeai as genai
from datetime import datetime, timedelta
import json

# Set the GEMINI_API_KEY
os.environ['API_KEY'] = userdata.get('API_KEY')

# Configure Google Gemini API
genai.configure(api_key=os.environ["API_KEY"])

# Set generation configuration
generation_config = {
    "temperature": 0.5,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 8192,
    "response_mime_type": "application/json",
}

# Create the model
model = genai.GenerativeModel(
  model_name="gemini-1.5-pro",
  generation_config=generation_config,
  # safety_settings = Adjust safety settings
  # See https://ai.google.dev/gemini-api/docs/safety-settings
  system_instruction="The application should be able to extract the following information from user queries:\n- Entity: The company name mentioned in the query (e.g., Flipkart, Amazon).\n- Parameter: The performance metric mentioned in the query (e.g., GMV, revenue, profit).\n- Start Date: The start date of the time period for which the metric is requested.\n- End Date: The end date of the time period for which the metric is requested.\n1. If the user query does not explicitly mention the start date and/or end date, assume the following defaults:\n- Start Date: Today's date minus one year.\n- End Date: Today's date.\n2. The extracted information should be converted into a JSON format with the following structure:\n[\n  {\n    \"entity\": \"Flipkart\",\n    \"parameter\": \"GMV\",\n    \"startDate\": \"2023-05-02\",\n    \"endDate\": \"2024-05-02\"\n  },\n  {\n    \"entity\": \"Amazon\",\n    \"parameter\": \"GMV\",\n    \"startDate\": \"2023-05-02\",\n    \"endDate\": \"2024-05-02\"\n  }\n]\n3. If the user query mentions multiple companies or requests a comparison, the JSON output should include multiple objects, one for each company mentioned.\n4. The start date and end date should be converted to the ISO 8601 format (YYYY-MM-DD) before including them in the JSON output.\nAdditional Considerations:\n- Handle variations in user queries, such as different spellings or abbreviations of company names and metric parameters.\n- Consider adding support for additional date formats or relative date ranges (e.g., \"last quarter\", \"previous month\").",
)

# Get the response from the model
def process_query(user_query):
  chat_session = model.start_chat(
    history=[
    {
      "role": "user",
      "parts": [
        "get me Flipkart's GMV for the last one year",
      ],
    },
    {
      "role": "model",
      "parts": [
        "```json\n[{\"entity\": \"Flipkart\", \"parameter\": \"GMV\", \"startDate\": \"2023-05-03\", \"endDate\": \"2024-05-03\"}]\n\n```",
      ],
    },
    {
      "role": "user",
      "parts": [
        "get me Amazon's GMV",
      ],
    },
    {
      "role": "model",
      "parts": [
        "```json\n[{\"entity\": \"Amazon\", \"parameter\": \"GMV\", \"startDate\": \"2023-05-03\", \"endDate\": \"2024-05-03\"}]\n\n```",
      ],
    },
    {
      "role": "user",
      "parts": [
        "compare both",
      ],
    },
    {
      "role": "model",
      "parts": [
        "```json\n[{\"entity\": \"Flipkart\", \"parameter\": \"GMV\", \"startDate\": \"2023-05-03\", \"endDate\": \"2024-05-03\"}, {\"entity\": \"Amazon\", \"parameter\": \"GMV\", \"startDate\": \"2023-05-03\", \"endDate\": \"2024-05-03\"}]\n\n```",
      ],
    },
  ]
)

  response = chat_session.send_message(user_query)

  # Parse and return the JSON from the response
  response_json = json.loads(response.text)  # Assuming response.text is JSON formatted
  return response_json

# Example user query
user_query = input()

# Process the query and print the result
result = process_query(user_query)
print(json.dumps(result, indent=2))


Get me Amazon's, Flipkart's and Myntra's GMV
[
  {
    "entity": "Amazon",
    "parameter": "GMV",
    "startDate": "2023-05-03",
    "endDate": "2024-05-03"
  },
  {
    "entity": "Flipkart",
    "parameter": "GMV",
    "startDate": "2023-05-03",
    "endDate": "2024-05-03"
  },
  {
    "entity": "Myntra",
    "parameter": "GMV",
    "startDate": "2023-05-03",
    "endDate": "2024-05-03"
  }
]
